In [1]:
"""
attaches genuine functionality onto the datastructures / UI elements defined in ui_*.py
"""
%run __init__.py
%load_ext lab_black

In [2]:
import pandas as pd
from IPython.display import (
    # update_display,
    display,
    Image,
    # JSON,
    Markdown,
    # HTML,
    clear_output,
)
import subprocess
import functools
from shutil import copyfile
import getpass
import importlib.util
from halo import HaloNotebook
import pathlib
import typing
from typing import Optional, List, Dict, Type
from pydantic.dataclasses import dataclass
from pydantic import BaseModel, validator, Field
from jinja2 import Template

import plotly.io as pio
import plotly.graph_objects as go

# widget stuff
import ipywidgets as widgets

# core mf_modules
from ipyautoui import AutoUi, DisplayFiles
from ipyautoui.autoui import display_template_ui_model

import inspect

# display_template_ui_model()

# from this repo
from ipyrun.utils import make_dir, del_matching
from ipyrun.constants import load_test_constants
from ipyrun.constants import (
    BUTTON_WIDTH_MIN,
    BUTTON_WIDTH_MEDIUM,
    JOBNO_DEFAULT,
    PATH_RUNAPP_HELP,
    PATH_RUNAPPS_HELP,
)

def get_mfuser_initials():
    user = getpass.getuser()
    return user[0] + user[2]

In [5]:
if __name__ == "__main__":
    ?AddRunDialogue

    def add_run_dialogue(cls=None):
        display(AddRunDialogue(cls))

    batch_actions = BatchActions(
        inputs_show=None,
        inputs_hide=None,
        outputs_show=None,
        outputs_hide=None,
        add_show=add_run_dialogue,
    )

    display(RunApps(batch_actions=batch_actions))

Init signature:
AddRunDialogue(
    app: Type[ipyrun.ui_add.RunApps] = None,
    add_cmd: Callable = <function create_runapp at 0x7fbe20604700>,
)
Docstring:      <no docstring>
Init docstring:
a ui element for adding new runs to RunApps

Args:
    app (RunApps): the app that will be modified by this class
    add_cmd (typing.Callable): a function that is executed to add a new run to the "app"
        on_click of a button. the main app is passed to the function using functools:

Code:
    ```
    def _init_controls(self):
        self.add.on_click(self._add)

    def _add(self, click):
        return functools.partial(self.add_cmd, cls=self.app)()
    ```
File:           /mnt/c/engDev/git_mf/ipyrun/src/ipyrun/ui_add.py
Type:           type
Subclasses:     


In [ ]:
# create a RunApp configuration
# because the RunApp defininition is very pluggable it can support many different types for configurations
# the one below is for the generic case of running a shell command
import io


class ShellHandler(BaseModel):
    call: str = "python -O"
    fpth_script: pathlib.Path
    fdir_appdata: pathlib.Path
    fpths_inputs: List[pathlib.Path]
    fpths_outputs: List[pathlib.Path]
    params: typing.Dict = {}
    fpth_config: pathlib.Path = "config-shell_handler.json"
    cmd_template: str = """\
{{ call }} {{ fpth_script }}\
{% for f in fpths_inputs %} {{f}}{% endfor %}\
{% for f in fpths_outputs %} {{f}}{% endfor %}\
{% for k,v in params.items()%} --{{k}} {{v}}{% endfor %}
"""
    cmd: str = ""
    fpth_runhistory: str = "runhistory.csv"
    # fpth_log: pathlib.Path

    @validator("fpth_config", always=True)
    def _fpth_config(cls, v, values):
        return values["fdir_appdata"] / v

    @validator("fpth_runhistory", always=True)
    def _fpth_runhistory(cls, v, values):
        return values["fdir_appdata"] / v

    @validator("cmd", always=True)
    def _cmd(cls, v, values):
        return Template(values["cmd_template"]).render(
            call=values["call"],
            fpth_script=values["fpth_script"],
            fpths_inputs=values["fpths_inputs"],
            fpths_outputs=values["fpths_outputs"],
            params=values["params"],
        )


class RunAppConfigShell(RunAppConfig):
    """this one is likely to be rewritten for each application"""

    config_actions: ShellHandler = None


def execute(cmd: str, cls=None):
    """
    cmd: str, cls=None
    """
    b = widgets.Button(
        layout={"width": BUTTON_WIDTH_MIN}, icon="fa-times", button_style="danger"
    )
    display(b)
    print(type(cls))

    def _b(click):
        with cls.out_console:
            clear_output()

    b.on_click(_b)
    cmd = cmd.split(" ")
    pr = "\n".join(cmd)
    pr = f"```{pr}```"
    display(Markdown(pr))
    spinner = HaloNotebook(animation="marquee", text="Running", spinner="dots")
    try:
        spinner.start()
        save = sys.stdout
        sys.stdout = io.StringIO()
        proc = subprocess.Popen(cmd)
        proc.wait()
        in_stdout = sys.stdout.getvalue()
        sys.stdout = save
        display(in_stdout)
        spinner.succeed("Finished")

    except subprocess.CalledProcessError as e:
        spinner.fail("Error with Process")


def create_inputs_file(
    model: typing.Type[BaseModel],
    index: int,
    fdir: pathlib.Path,
    process_name: str = "",
):
    input_type = model.schema()["title"].lower()
    name_inputs = f"inputs-{input_type}-{process_name}-{index}.lg.json"
    path_inputs = fdir / name_inputs
    model().file(path_inputs)  #  create inputs file
    return path_inputs


def default_runapp_config(
    path_script: pathlib.Path,
    index: int,
    input_models: typing.List[typing.Type[BaseModel]],
    process_name: str = None,
    fdir_appdata: pathlib.Path = None,
    class_displayfiles: typing.Type[DisplayFiles] = DisplayFiles,
) -> typing.Tuple[RunAppConfigShell, RunActions]:

    if fdir_appdata is None:
        fdir_appdata = path_script.parent

    if process_name is None:
        process_name = path_script.stem.replace("script_", "")

    run_id = RunId(process_name=process_name, check=True, index=index)
    paths_inputs = [
        create_inputs_file(model, index, fdir_appdata, process_name=process_name)
        for model in input_models
    ]

    config_actions = ShellHandler(
        fpth_script=path_script,
        fdir_appdata=path_script.parent,
        fpths_inputs=paths_inputs,
        fpths_outputs=[
            fdir_appdata / f"outputs-{run_id.process_name}-{run_id.index}.csv",
            fdir_appdata / f"outputs-{run_id.process_name}-{run_id.index}.plotly.json",
        ],
        # params={'k':'v'}
    )
    config_runapp = RunAppConfigShell(run_id=run_id, config_actions=config_actions)
    run_actions = RunActions(
        help_config_show=None,
        # help_ui_show=None,
        help_run_show=(
            lambda: display(class_displayfiles([config_actions.fpth_script]))
        ),
        inputs_show=(
            lambda: display(
                class_displayfiles(
                    [f for f in config_actions.fpths_inputs], auto_open=True
                )
            )
        ),
        outputs_show=(
            lambda: display(
                class_displayfiles(
                    [f for f in config_actions.fpths_outputs], auto_open=True
                )
            )
        ),
        run=functools.partial(execute, config_actions.cmd),
    )
    return config_runapp, run_actions


if __name__ == "__main__":
    display(
        Markdown(
            """
### RunAppConfigShell

Extend the `RunAppConfig`
    """
        )
    )
    display(Markdown("`>>> display(RunAppConfigShell())`"))
    config_runapp = RunAppConfigShell()
    display(RunAppConfigShell().dict())

In [ ]:
if __name__ == "__main__":
    from ipyrun.constants import load_test_constants
    from ipyautoui.autoui import AutoUi, AutoUiConfig
    from ipyautoui.displayfile import DisplayFiles, DisplayFile

    test_constants = load_test_constants()
    sys.path.append(str(test_constants.DIR_EXAMPLE_PROCESS))
    from schemas import LineGraph
    import functools
    import sys

    config_autoui = AutoUiConfig(pydantic_model=LineGraph, ext=".lg.json")
    LineGraphUi = AutoUi.create_displayfile(config_autoui)

    def line_graph_prev(path):
        display(LineGraphUi(path))

    user_file_renderers = {".lg.json": line_graph_prev}
    DisplayFiles = functools.partial(
        DisplayFiles, user_file_renderers=user_file_renderers
    )  # overwrite the DisplayFiles class with the .lg.json file renderer baked in
    test_constants = load_test_constants()

    PATH_SCRIPT = list(test_constants.DIR_EXAMPLE_PROCESS.glob(pattern="script*"))[0]
    tu = default_runapp_config(
        PATH_SCRIPT, 1, input_models=[LineGraph], class_displayfiles=DisplayFiles
    )
    run = RunApp.from_config(*tu)
    display(run)

In [ ]:
from io import StringIO


def create_runapp_linegraph(
    path_script=PATH_SCRIPT,
    input_models=[LineGraph],
    class_displayfiles=DisplayFiles,
    cls=None,
):
    if cls is not None:
        index = len(cls.apps)
    else:
        index = 0
    return RunApp.from_config(
        *default_runapp_config(
            path_script,
            index,
            input_models,
            class_displayfiles=class_displayfiles,
        )
    )


def add_linegraph_dialogue(cls=None):
    display(AddRunDialogue(cls, add_cmd=create_runapp_linegraph))


batch_actions = BatchActions(
    inputs_show=None,
    inputs_hide=None,
    outputs_show=None,
    outputs_hide=None,
    add_show=add_linegraph_dialogue,
)
batch = RunApps(batch_actions=batch_actions, apps=[create_runapp_linegraph()])
display(batch)
# create_runapp_linegraph(2)